In [ ]:
import math
import numpy as np
import librosa
from obspy import Trace
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.image as img
import matplotlib
import sys
import obspy
import pickle as pkl
import glob as g
from obspy.clients.fdsn import Client
import datetime
import re
import math
from numpy import linalg as eigen
# import noisecut

sys.path.insert(1,'/Users/charlesh/Documents/Codes/')
import matplotlib.colors as mcolors
from scipy.stats import norm
import scipy.stats as stats
from scipy import fft
# import ntk
from cmcrameri import cm

# ------------------------------------
sys.path.append('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS')
sys.path.append('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS')
import ObsQA
import ObsQA as ob
from ObsQA.plots import qtp
from ObsQA import *
sys.path.append('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/OBStools')
import obstools as obs
import cmath
from comp_tools import *
import ObsQA.metrics as metrics

NoiseFolder = '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE'
CompFolder = NoiseFolder + '/COMPS/ATaCR_NC'
MethodsFolder = NoiseFolder + '/METHODS/ATaCR'

In [ ]:
keys = list(cm.cmaps.keys())
# keys = ['acton_categorical','glasgow_categorical','hawaii_categorical','lipari_categorical']
# keys = ['batlow_categorical','davos_categorical','acton_categorical','imola_categorical','navia_categorical','bilbao_categorical']

# raw_palette = 'grayC_categorical'
# atacr_palette = 'bilbao_categorical'
# nc_palette = 'devon_categorical'
# comp_palette = 'lajolla_categorical'
# keys = [raw_palette,atacr_palette,nc_palette]

for key in keys:
    if key.endswith('_categorical'):
        display(cm.cmaps[key])

In [ ]:
# Load catalog
plotfolder = NoiseFolder + '/COMPS/ObsQA/figures'
compfolder = CompFolder
ATaCR_Py_DataFolder = ob.io.dir_libraries(MethodsFolder)[1]
datafolder = ATaCR_Py_DataFolder['Py_DataParentFolder']
eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
catalog = pd.read_pickle(eventsfolder + '/event_catalog.pkl')
catalog['Network_Experiment'] = '[' + catalog.Network + '] ' + catalog.Experiment
catalog['StaName'] = catalog.Network + '.' + catalog.Station
# display(catalog)

In [ ]:
catalog = ob.io.getstalist()[0]
# catalog

In [ ]:
colors = cm.cmaps['batlow'].colors
colors = [mcolors.to_hex(c) for c in colors]

In [ ]:
colors = cm.cmaps['hawaii_categorical'].colors
colors = [mcolors.to_hex(c) for c in colors]

# cmaps = get_obs_cmaps(extrema_n=0)
# colors = cmaps['comp']

catalog = catalog.sort_values(by='Network')

clr = dict()
for i,k in enumerate(np.unique(catalog.Network.to_list())):
    clr[k] = colors[i+3]

plt.figure(figsize=(12,6))
StartX = catalog.Start.to_list()
EndY = catalog.End.to_list()
Labels = catalog.StaName.to_list()
Labels = '[' + catalog.Network +  '] ' + catalog.Experiment
Nets = catalog.Network.to_list()
for x,y,l,n in zip(StartX,EndY,Labels,Nets):
    plt.plot_date([x,y],[l,l],ls='-',c=clr[n])
plt.title('Data Availability')
# plt.gca().get_xlim()
plt.grid(which='both')
plt.xlim(12000,19000)
save_tight('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS/ObsQA/figures/metrics/All_DataAvailability.png')

In [ ]:
# tfavg='sta'
# # catalog = catalog.iloc[35:]
# for stai,Station in enumerate(catalog.iloc):
#         for evi,Event in enumerate(Station.Events):
#                 # [LOAD EVENT]----------------------
#                 staname = Station.StaName
#                 datename = Station.Metadata[evi].events[0].origins[0].time.strftime('%Y.%j')
#                 notify = '[' + staname + '] || ' + 'S: [' + str(stai+1) + '/' + str(len(catalog)) + '] |  E: [' + str(evi+1) + '/' + str(len(Station.Events)) + ']' + ' || ' +  datename
#                 try:
#                         d = ob.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[Event],Station.Network,Station.Station,tfavg=tfavg)
#                 except:
#                         print(notify + ' :: Corrected event not found. Skipping.')
#                         continue
#                 print(notify)
#                 # [NOISE METRICS]----------------------
#                 DN = ob.io.get_DayNoise(datafolder,d.Network[0],d.Station[0])
#                 f,spectra = DNSpectra(DN)
#                 COH,PH,ADM = metrics.obmetrics(spectra)
#                 # [COLLECT DATA]----------------------
#                 # meter = [metrics.CrossCOH,metrics.CrossPh,metrics.CrossAdm]
#                 meter = [metrics.CrossCOH,metrics.CrossPh,metrics.CrossAdm]
#                 # meter = [metrics.CrossCOH]
#                 RawP,RawZ,PostATACR,PostHPS,PostBoth_Z21 = get_ATACR_HPS_Comp(d,atacr_TFs_used='ZP-21')
#                 title = [k.stats.location for k in [PostATACR,PostHPS,PostBoth]]
#                 GreyComp = RawP
#                 PreCollect = []
#                 PostCollect = []
#                 for i,func in enumerate(meter):
#                         f, Raw_Metric = func(RawZ,GreyComp)
#                         f, post_atacr_metric = func(PostATACR,GreyComp)
#                         f, post_hps_metric = func(PostHPS,GreyComp)
#                         f, post_both_metric = func(PostBoth,GreyComp)
#                         pre = [Raw_Metric,Raw_Metric,Raw_Metric]
#                         post = [post_atacr_metric,post_hps_metric,post_both_metric]
#                         x = f
#                         PreCollect.append(pre)
#                         PostCollect.append(post)
#                 # [PLOT]----------------------------------------------------------------------
#                 ylabels = ['Coherence','Phase (Deg.)','Admittance']
#                 yscale = ['linear','linear','log']
#                 style = ['scatter','scatter','scatter']
#                 bg = [[COH['f'],COH['ZP']],[PH['f'],PH['ZP']],[ADM['f'],ADM['ZP']]]

#                 magname = 'm' + str(Station.Metadata[evi].events[0].magnitudes[0].mag)
#                 fig_title = [staname + ' | ' + datename + '.' + magname,1] #figure title and column to plot it
#                 outfolder = plotfolder + '/metrics/' 'Using_' + tfavg.upper() + '_AvgTFs'
#                 fileout = outfolder + '/' + staname + '.' + datename + '.' + magname + '.' + tfavg +'.png'
#                 pre_label = 'Raw'
#                 post_label = 'Corrected'
#                 bg_label = 'Noise'
#                 ObsQA.plots.CompPlot(x,PreCollect,PostCollect,style=style,title = title,xlabel='Hz',ylabel=ylabels,xscale='log',yscale=yscale,alpha=(0.1,0.6),color=('r','b'),figsize=(15,6),s=[.1,3],group_title=fig_title,bg=bg,bg_label = bg_label,pre_label=pre_label,post_label=post_label)
#                 try:
#                         os.mkdir(outfolder)
#                 except:
#                         pass
#                 save_tight(fileout,dpi=300)

In [ ]:
# # Build PDFs of spectra

# # Get event and day data
# stas = ['LA33']
# evi = 0
# # stas = ['FN07A','FN12C','FN14A','FS15B','G03A','J59C','LA33','LA34']
# # ---------------------------------------------------------------------------------------------------------------------------------------------
# sta = catalog[np.in1d(catalog.Station,stas)].iloc[0]
# d = obs.utilities.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[evi]],sta.Network,sta.Station,tfavg='sta')
# DN = ob.io.get_DayNoise(datafolder,d.Network[0],d.Station[0])

# f,spectra = DNSpectra(DN)
# pdf = SpecPDF(spectra)

# ncols = 2
# nrows = 5
# fig, axes = plt.subplots(nrows=nrows, ncols=ncols,figsize=(25,20),sharex='all',sharey='all')
# axes = axes.reshape(-1)
# # cmap = plt.colormaps.get_cmap('tab20').colors
# cmap = plt.colormaps.get_cmap('gist_rainbow_r')
# cmap.set_under(color='white')
# for i,key in enumerate(spectra.keys()):
#     y = np.abs(spectra[key])
#     y = 10*np.log10(y) #power to decibels
#     x = np.ones(y.shape)*f
#     z = pdf[key]

#     trendx = f
#     trendy = y[np.argmin(np.abs(z - z.max(axis=0)),axis=0),np.arange(z.shape[1])]
#     trendx,trendy = log_smoothing(f,trendy)

#     axes[i].scatter(x,y,label=key,s=0.1,c=z,cmap=cmap,vmin=0)
#     axes[i].scatter(trendx,trendy,s=0.4,c='k',label='Peak')
#     axes[i].legend(loc='lower left',markerscale=15)
#     axes[i].set_xscale('log')
#     plt.colorbar(mappable=axes[i].collections[0])
#     if i<=ncols:
#         axes[i].set_title(d.Network[0] + '.' + d.Station[0])

In [ ]:
# # ---
# # =================================================================================
# TFs = 'ZP-21'
# RawP,RawZ,PostATACR,PostHPS,PostBoth = get_ATACR_HPS_Comp(d,atacr_TFs_used=TFs)
# tr1 = d.Raw[0]['tr1']
# tr2 = d.Raw[0]['tr2']
# fs = PostATACR.stats.sampling_rate
# f,raw_zp_coh = CrossCOH(RawZ,RawP,fs=fs)
# f,raw_1p_coh = CrossCOH(tr1,RawP,fs=fs)
# f,raw_2p_coh = CrossCOH(tr2,RawP,fs=fs)

# f,atacr_zp_coh = CrossCOH(PostATACR,RawP,fs=fs)
# f,hps_zp_coh = CrossCOH(PostHPS,RawP,fs=fs)
# f,atacr_nc_coh = CrossCOH(PostATACR,PostHPS,fs=fs)
# # --

# fig, axes = plt.subplots(nrows=2, ncols=1,figsize=(15,10),sharex='all',sharey='all')
# axes = np.atleast_2d(axes).T

# axes[0,0].scatter(f,raw_zp_coh,s=0.02*50,label='RAW | ZP Coherence',alpha=1.0)
# axes[0,0].scatter(f,raw_1p_coh,s=0.02*50,label='RAW | 1P Coherence',alpha=0.7)
# axes[0,0].scatter(f,raw_2p_coh,s=0.02*50,label='RAW | 2P Coherence',alpha=0.4)
# axes[0,0].legend(markerscale=10)
# axes[0,0].set_ylabel('Coherence')
# axes[0,0].set_xscale('log')

# axes[1,0].scatter(f,atacr_zp_coh,s=1,label='ATaCR | ZP Coherence',c='r')
# axes[1,0].scatter(f,hps_zp_coh,s=1,label='HPS | ZP Coherence',c='b')
# axes[1,0].scatter(f,atacr_nc_coh,s=1,label='ATaCR (' + TFs + ') | HPS Coherence',c='grey')
# axes[1,0].legend(markerscale=10)
# axes[1,0].set_xlim(fs/atacr_nc_coh.size,2.5)
# axes[1,0].set_ylim(-0.01,1.01)
# axes[1,0].set_ylabel('Coherence')
# axes[1,0].set_xlabel('Hz')
# axes[1,0].set_xscale('log')
# # =================================================================================
# # ---
# magname = 'm' + str(Station.Metadata[evi].events[0].magnitudes[0].mag)
# fig_title = [staname + ' | ' + datename + '.' + magname,1] #figure title and column to plot it
# axes[0,0].set_title(fig_title[0])
# outfolder = plotfolder + '/metrics/' 'ATaCR_HPS_COH/not_normed/' + tfavg
# fileout = outfolder + '/' + staname + '.' + datename + '.' + magname + '.' + tfavg +'.png'

In [ ]:
# colors = cm.cmaps[p].colors
# colors = cm.cmaps[p].colors[((cm.cmaps[p].colors**2).sum(axis=1)**(0.5))<0.9]
# colors = colors[(colors**2).sum(axis=1)**(0.5)>0.4]
# n = 5
# cmaps = dict()
# atacr = [1,0,0]
# nc = [0,0,1]
# comp = [0,1,0]
# raw = [0,0,0]

# cols = [atacr,nc,comp,raw]
# clr = cols[0]

# for k,p in zip(['atacr','nc','both','raw'],cols):
#     colors = [p]
#     clrstep = (p + (1-np.mean(p))*0.5)
#     clrstep[clrstep>1.0] = 1.0
#     colors.append(clrstep)
#     for _ in range(n):
#         colors.append(colors[-1]**0.9)
#     cmaps[k] = colors
# cmaps

In [ ]:
# cmaps = get_obs_cmaps(extrema_n=1)
# tfavg='day'
# stai = 0
# evi = 0
# Station  = catalog.iloc[stai]
# Event =  Station.Events[evi]

In [ ]:
# # [LOAD EVENT]----------------------
# staname = Station.StaName
# datename = Station.Metadata[evi].events[0].origins[0].time.strftime('%Y.%j')
# notify = '[' + staname + '] || ' + 'S: [' + str(stai+1) + '/' + str(len(catalog)) + '] |  E: [' + str(evi+1) + '/' + str(len(Station.Events)) + ']' + ' || ' +  datename
# try:
#         d = ob.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[Event],Station.Network,Station.Station,tfavg=tfavg)
# except:
#         print(notify + ' :: Corrected event not found. Skipping.')
# print(notify)

# # ---
# # =================================================================================
# # ---

# RawP,RawZ,PostATACR_Z21,PostHPS,PostBoth_Z21 = get_ATACR_HPS_Comp(d,atacr_TFs_used='ZP-21')
# _,_,_,_,PostBoth_ZPH = get_ATACR_HPS_Comp(d,atacr_TFs_used='ZP-H')

# PostATACR_ZPH = d.Corrected[0]['ZP-H'].copy()
# tr1 = d.Raw[0]['tr1']
# tr2 = d.Raw[0]['tr2']
# fs = PostATACR_Z21.stats.sampling_rate

# f,raw_zp_coh = CrossCOH(RawZ,RawP,fs=fs)
# f,raw_1p_coh = CrossCOH(tr1,RawP,fs=fs)
# f,raw_z1_coh = CrossCOH(tr1,RawZ,fs=fs)
# f,raw_z2_coh = CrossCOH(tr2,RawZ,fs=fs)
# f,raw_2p_coh = CrossCOH(tr2,RawP,fs=fs)

# f,atacr_z21_raw_p_coh = CrossCOH(PostATACR_Z21,RawP,fs=fs)
# f,atacr_z21_raw_z_coh = CrossCOH(PostATACR_Z21,RawZ,fs=fs)
# f,atacr_zph_raw_p_coh = CrossCOH(PostATACR_ZPH,RawP,fs=fs)
# f,atacr_zph_raw_z_coh = CrossCOH(PostATACR_ZPH,RawZ,fs=fs)

# f,hps_z_raw_p_coh = CrossCOH(PostHPS,RawP,fs=fs)
# f,hps_z_raw_z_coh = CrossCOH(PostHPS,RawZ,fs=fs)

# f,atacr_z21_hps_z_coh = CrossCOH(PostATACR_Z21,PostHPS,fs=fs)
# f,atacr_zph_hps_z_coh = CrossCOH(PostATACR_ZPH,PostHPS,fs=fs)

# f,both_z21_z_raw_z_coh = CrossCOH(PostBoth_Z21,RawZ,fs=fs)
# f,both_z21_z_raw_p_coh = CrossCOH(PostBoth_Z21,RawP,fs=fs)

# f,both_zph_z_raw_z_coh = CrossCOH(PostBoth_ZPH,RawZ,fs=fs)
# f,both_zph_z_raw_p_coh = CrossCOH(PostBoth_ZPH,RawP,fs=fs)
# # --

# f,both_zph_z_atacr_zph_coh = CrossCOH(PostBoth_ZPH,PostATACR_ZPH,fs=fs)
# f,both_z21_z_atacr_zph_coh = CrossCOH(PostBoth_Z21,PostATACR_ZPH,fs=fs)
# f,both_zph_z_atacr_z21_coh = CrossCOH(PostBoth_ZPH,PostATACR_Z21,fs=fs)
# f,both_z21_z_atacr_z21_coh = CrossCOH(PostBoth_Z21,PostATACR_Z21,fs=fs)
# f,both_zph_z_hps_z_coh = CrossCOH(PostBoth_ZPH,PostHPS,fs=fs)
# f,both_z21_z_hps_z_coh = CrossCOH(PostBoth_Z21,PostHPS,fs=fs)

# f,both_zph_z_raw_z_coh = CrossCOH(PostBoth_Z21,RawZ,fs=fs)

# fig, axes = plt.subplots(nrows=4, ncols=1,figsize=(14,14),sharex='all',sharey='all')
# axes = np.atleast_2d(axes).T

# axes[0,0].scatter(f,raw_zp_coh,s=0.02*50,label='ZP',alpha=0.5,c=cmaps['raw'][0])
# axes[0,0].scatter(f,raw_1p_coh,s=0.02*50,label='P1',alpha=0.1,c=cmaps['raw'][1])
# axes[0,0].scatter(f,raw_2p_coh,s=0.02*50,label='P2',alpha=0.1,c=cmaps['raw'][2])
# axes[0,0].scatter(f,raw_z1_coh,s=0.02*50,label='Z1',alpha=0.1,c=cmaps['raw'][3])
# axes[0,0].scatter(f,raw_z2_coh,s=0.02*50,label='Z2',alpha=0.1,c=cmaps['raw'][4])

# axes[1,0].scatter(f,hps_z_raw_p_coh,s=4,label='P | HPS',c=cmaps['nc'][6],alpha=0.5)
# axes[1,0].scatter(f,atacr_zph_raw_p_coh,s=2,label='P | ATaCR (ZP-H)',c=cmaps['atacr'][3],alpha=0.5)
# axes[1,0].scatter(f,atacr_z21_raw_p_coh,s=4,label='P | ATaCR (' + 'ZP-21' + ')',c=cmaps['atacr'][2],alpha=0.5)
# axes[1,0].scatter(f,both_zph_z_raw_p_coh,s=1,label='P | ATaCR (' + 'ZP-H' + ') w/ HPS',c=cmaps['both'][1])
# axes[1,0].scatter(f,both_z21_z_raw_p_coh,s=1,label='P | ATaCR (' + 'ZP-21' + ') w/ HPS',c=cmaps['both'][0])

# # axes[1,0].scatter(f,raw_zp_coh,s=0.02*50,label='RAW | ZP Coherence',alpha=1.0,c='k')
# axes[2,0].scatter(f,hps_z_raw_z_coh,s=1,label='Z | HPS',c=cmaps['nc'][6])
# axes[2,0].scatter(f,atacr_zph_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-H' + ')',c=cmaps['atacr'][3])
# axes[2,0].scatter(f,atacr_z21_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-21' + ')',c=cmaps['atacr'][2])
# axes[2,0].scatter(f,both_zph_z_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-H' + ') w/ HPS',c=cmaps['both'][1])
# axes[2,0].scatter(f,both_z21_z_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-21' + ') w/ HPS',c=cmaps['both'][0])


# axes[3,0].scatter(f,atacr_zph_hps_z_coh,s=.5,label='ATaCR (ZP-H) | HPS',c=cmaps['comp'][7])
# axes[3,0].scatter(f,atacr_z21_hps_z_coh,s=.5,label='ATaCR (ZP-21) | HPS',c=cmaps['comp'][8])
# axes[3,0].scatter(f,both_zph_z_atacr_zph_coh,s=.5,label='ATaCR (ZP-H) | HPS w/ ATaCR (' + 'ZP-H' + ')',c=cmaps['both'][4])
# axes[3,0].scatter(f,both_z21_z_atacr_z21_coh,s=.5,label='ATaCR (ZP-21) | HPS w/ ATaCR (' + 'ZP-21' + ')',c=cmaps['both'][8])

# for i in range(axes.shape[0]):
#         # axes[i,0].legend(markerscale=10,loc='upper left')
#         axes[i,0].set_ylabel('Coherence')
#         axes[i,0].set_xscale('log')
#         axes[i,0].set_xlim(fs/atacr_nc_coh.size,2.5)
#         axes[i,0].set_ylim(-0.01,1.01)
#         axes[i,0].grid(which='major',alpha=0.5)

# axes[-1,0].set_xlabel('Hz')
# # =================================================================================
# magname = 'm' + str(Station.Metadata[evi].events[0].magnitudes[0].mag)
# fig_title = [staname + ' | ' + datename + '.' + magname,1] #figure title and column to plot it....0.........0..............................0.
# outfolder = plotfolder + '/metrics/' 'ATaCR_HPS_COH/' + tfavg
# fileout = outfolder + '/' + staname + '.' + datename + '.' + magname + '.' + tfavg +'.png'
# axes[0,0].set_title(fig_title[0] + '\n Raw Data Component Coherence')
# axes[1,0].set_title('Pressure Coherence after Corrections (ZP)')
# axes[2,0].set_title('Raw Vertical Coherence after Corrections (ZZ)')
# axes[3,0].set_title('Vertical Coherence Between Methods (ZZ)')
# # =================================================================================
# for k in range(4):
#         lg = axes[k,0].legend(markerscale=10,loc='upper left')
#         for i in range(len(lg.legendHandles)):
#                 lg.legendHandles[i]._sizes = np.array([70])
#                 lg.legendHandles[i]._alpha = 1.0
# # =================================================================================
# try:
#         os.mkdir(outfolder)
# except:
#         pass
# # save_tight(fileout,dpi=300)

In [ ]:
# cmaps = get_obs_cmaps(extrema_n=1)

# tfavg='day'
# # catalog = catalog[np.in1d(catalog.Station,'M08A')]
# for stai,Station in enumerate(catalog.iloc):
#         for evi,Event in enumerate(Station.Events):
# # -------------# -------------# -------------# -------------
#                 # [LOAD EVENT]----------------------
#                 staname = Station.StaName
#                 datename = Station.Metadata[evi].events[0].origins[0].time.strftime('%Y.%j')
#                 notify = '[' + staname + '] || ' + 'S: [' + str(stai+1) + '/' + str(len(catalog)) + '] |  E: [' + str(evi+1) + '/' + str(len(Station.Events)) + ']' + ' || ' +  datename
#                 try:
#                         d = ob.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[Event],Station.Network,Station.Station,tfavg=tfavg)
#                 except:
#                         print(notify + ' :: Corrected event not found. Skipping.')
#                         continue
#                 print(notify)

#                 # ---
#                 # =================================================================================
#                 # ---

#                 RawP,RawZ,PostATACR_Z21,PostHPS,PostBoth_Z21 = get_ATACR_HPS_Comp(d,atacr_TFs_used='ZP-21')
#                 _,_,_,_,PostBoth_ZPH = get_ATACR_HPS_Comp(d,atacr_TFs_used='ZP-H')

#                 PostATACR_ZPH = d.Corrected[0]['ZP-H'].copy()
#                 tr1 = d.Raw[0]['tr1']
#                 tr2 = d.Raw[0]['tr2']
#                 fs = PostATACR_Z21.stats.sampling_rate

#                 f,raw_zp_coh = CrossCOH(RawZ,RawP,fs=fs)
#                 f,raw_1p_coh = CrossCOH(tr1,RawP,fs=fs)
#                 f,raw_z1_coh = CrossCOH(tr1,RawZ,fs=fs)
#                 f,raw_z2_coh = CrossCOH(tr2,RawZ,fs=fs)
#                 f,raw_2p_coh = CrossCOH(tr2,RawP,fs=fs)

#                 f,atacr_z21_raw_p_coh = CrossCOH(PostATACR_Z21,RawP,fs=fs)
#                 f,atacr_z21_raw_z_coh = CrossCOH(PostATACR_Z21,RawZ,fs=fs)
#                 f,atacr_zph_raw_p_coh = CrossCOH(PostATACR_ZPH,RawP,fs=fs)
#                 f,atacr_zph_raw_z_coh = CrossCOH(PostATACR_ZPH,RawZ,fs=fs)

#                 f,hps_z_raw_p_coh = CrossCOH(PostHPS,RawP,fs=fs)
#                 f,hps_z_raw_z_coh = CrossCOH(PostHPS,RawZ,fs=fs)

#                 f,atacr_z21_hps_z_coh = CrossCOH(PostATACR_Z21,PostHPS,fs=fs)
#                 f,atacr_zph_hps_z_coh = CrossCOH(PostATACR_ZPH,PostHPS,fs=fs)

#                 f,both_z21_z_raw_z_coh = CrossCOH(PostBoth_Z21,RawZ,fs=fs)
#                 f,both_z21_z_raw_p_coh = CrossCOH(PostBoth_Z21,RawP,fs=fs)

#                 f,both_zph_z_raw_z_coh = CrossCOH(PostBoth_ZPH,RawZ,fs=fs)
#                 f,both_zph_z_raw_p_coh = CrossCOH(PostBoth_ZPH,RawP,fs=fs)
#                 # --

#                 f,both_zph_z_atacr_zph_coh = CrossCOH(PostBoth_ZPH,PostATACR_ZPH,fs=fs)
#                 f,both_z21_z_atacr_zph_coh = CrossCOH(PostBoth_Z21,PostATACR_ZPH,fs=fs)
#                 f,both_zph_z_atacr_z21_coh = CrossCOH(PostBoth_ZPH,PostATACR_Z21,fs=fs)
#                 f,both_z21_z_atacr_z21_coh = CrossCOH(PostBoth_Z21,PostATACR_Z21,fs=fs)
#                 f,both_zph_z_hps_z_coh = CrossCOH(PostBoth_ZPH,PostHPS,fs=fs)
#                 f,both_z21_z_hps_z_coh = CrossCOH(PostBoth_Z21,PostHPS,fs=fs)

#                 f,both_zph_z_raw_z_coh = CrossCOH(PostBoth_Z21,RawZ,fs=fs)

#                 fig, axes = plt.subplots(nrows=4, ncols=1,figsize=(14,14),sharex='all',sharey='all')
#                 axes = np.atleast_2d(axes).T

#                 axes[0,0].scatter(f,raw_zp_coh,s=0.02*50,label='ZP',alpha=0.5,c=cmaps['raw'][0])
#                 axes[0,0].scatter(f,raw_1p_coh,s=0.02*50,label='P1',alpha=0.1,c=cmaps['raw'][1])
#                 axes[0,0].scatter(f,raw_2p_coh,s=0.02*50,label='P2',alpha=0.1,c=cmaps['raw'][2])
#                 axes[0,0].scatter(f,raw_z1_coh,s=0.02*50,label='Z1',alpha=0.1,c=cmaps['raw'][3])
#                 axes[0,0].scatter(f,raw_z2_coh,s=0.02*50,label='Z2',alpha=0.1,c=cmaps['raw'][4])

#                 axes[1,0].scatter(f,hps_z_raw_p_coh,s=4,label='P | HPS',c=cmaps['nc'][6],alpha=0.5)
#                 axes[1,0].scatter(f,atacr_zph_raw_p_coh,s=2,label='P | ATaCR (ZP-H)',c=cmaps['atacr'][3],alpha=0.5)
#                 axes[1,0].scatter(f,atacr_z21_raw_p_coh,s=4,label='P | ATaCR (' + 'ZP-21' + ')',c=cmaps['atacr'][2],alpha=0.5)
#                 axes[1,0].scatter(f,both_zph_z_raw_p_coh,s=1,label='P | ATaCR (' + 'ZP-H' + ') w/ HPS',c=cmaps['both'][1])
#                 axes[1,0].scatter(f,both_z21_z_raw_p_coh,s=1,label='P | ATaCR (' + 'ZP-21' + ') w/ HPS',c=cmaps['both'][0])

#                 # axes[1,0].scatter(f,raw_zp_coh,s=0.02*50,label='RAW | ZP Coherence',alpha=1.0,c='k')
#                 axes[2,0].scatter(f,hps_z_raw_z_coh,s=1,label='Z | HPS',c=cmaps['nc'][6])
#                 axes[2,0].scatter(f,atacr_zph_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-H' + ')',c=cmaps['atacr'][3])
#                 axes[2,0].scatter(f,atacr_z21_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-21' + ')',c=cmaps['atacr'][2])
#                 axes[2,0].scatter(f,both_zph_z_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-H' + ') w/ HPS',c=cmaps['both'][1])
#                 axes[2,0].scatter(f,both_z21_z_raw_z_coh,s=1,label='Z | ATaCR (' + 'ZP-21' + ') w/ HPS',c=cmaps['both'][0])


#                 axes[3,0].scatter(f,atacr_zph_hps_z_coh,s=.5,label='ATaCR (ZP-H) | HPS',c=cmaps['comp'][7])
#                 axes[3,0].scatter(f,atacr_z21_hps_z_coh,s=.5,label='ATaCR (ZP-21) | HPS',c=cmaps['comp'][8])
#                 axes[3,0].scatter(f,both_zph_z_atacr_zph_coh,s=.5,label='ATaCR (ZP-H) | HPS w/ ATaCR (' + 'ZP-H' + ')',c=cmaps['both'][4])
#                 axes[3,0].scatter(f,both_z21_z_atacr_z21_coh,s=.5,label='ATaCR (ZP-21) | HPS w/ ATaCR (' + 'ZP-21' + ')',c=cmaps['both'][8])

#                 for i in range(axes.shape[0]):
#                         # axes[i,0].legend(markerscale=10,loc='upper left')
#                         axes[i,0].set_ylabel('Coherence')
#                         axes[i,0].set_xscale('log')
#                         axes[i,0].set_xlim(fs/atacr_zph_hps_z_coh.size,2.5)
#                         axes[i,0].set_ylim(-0.01,1.01)
#                         axes[i,0].grid(which='major',alpha=0.5)

#                 axes[-1,0].set_xlabel('Hz')
#                 # =================================================================================
#                 magname = 'm' + str(Station.Metadata[evi].events[0].magnitudes[0].mag)
#                 fig_title = [staname + ' | ' + datename + '.' + magname,1] #figure title and column to plot it....0.........0..............................0.
#                 outfolder = plotfolder + '/metrics/' 'ATaCR_HPS_COH/' + tfavg
#                 fileout = outfolder + '/' + staname + '.' + datename + '.' + magname + '.' + tfavg +'.png'
#                 axes[0,0].set_title(fig_title[0] + '\n Raw Data Component Coherence')
#                 axes[1,0].set_title('Pressure Coherence after Corrections (ZP)')
#                 axes[2,0].set_title('Raw Vertical Coherence after Corrections (ZZ)')
#                 axes[3,0].set_title('Vertical Coherence Between Methods (ZZ)')
#                 # =================================================================================
#                 for k in range(4):
#                         lg = axes[k,0].legend(markerscale=10,loc='upper left')
#                         for i in range(len(lg.legendHandles)):
#                                 lg.legendHandles[i]._sizes = np.array([70])
#                                 lg.legendHandles[i]._alpha = 1.0
#                 # =================================================================================
#                 try:
#                         os.mkdir(outfolder)
#                 except:
#                         pass
#                 save_tight(fileout,dpi=300)